# HL7 to JSON converter

### Rakesh Gupta MSDS 5023

### October 8th, 2015

This is a simple program that reads an HL7 file and prints it out to the output in a JSON format. The program has two functions

1. printJsonSecondLevel - 
    This takes a message and a message key as the input parameters. the message is split into an array using a ^ as the splitter function. The array is looped through to convert into a json string which is returned back to the calling program.
    

2. printJsonFirstlevel - 
    This takes a message as the input parameters and then it is split into an array on the "|" as the splitter function. The array is looped through to convert into a json string which is returned back to the calling program.
    This program calls the printJsonSecondLevel function
    
3. The last section of the code reads the file that was opened, splits the data into an array and goes through the file calling printJsonFirstLevel


Future improvements
-  Add a printJsonThirdLevel to split based on the "~".
-  See if the various printJson functions can be generalised and called recursively on itself to do the printJsonSecondLevel and printJsonThirdLevel


In [1]:
import numpy as np
import pandas as pd 
#import simplejson

In [2]:
filename = 'C:\\rakesh\\InformationStructures\\msg1.txt'

In [3]:
current_file = open(filename, mode = 'U')
data = current_file.read()
print data

MSH|^~\&||ABCDE|||201507230021||ADT^A08|CAGTADM.1.10532994|P|2.1
EVN|A08|201507230021|||R.CA.NAH^HALL^NANCY^A^^^
PID|1||V000042610|M88604|BARTON^CLINTON^FRANCIS^^^||19510121|M|^^^^^|W|9 E CARTER ST APT A^^CARTERSVILLE^GA^30120^USA^^^BARTOW||(000)000-0000|(000)000-0000|ENG|D|OTH|V07016760770|000-00-0000
NK1|1|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floating Island^^CARTERSVILLE^GA^30120^USA^^^BARTOW|(000)000-0000|(000)000-0000
NK1|2|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floating Island^^CARTERSVILLE^GA^30120^USA^^^BARTOW|(000)000-0000|(000)000-0000
PV1|1|E|V.ER^^|EM|||WILSON^Wade^Deadpool^^^MD|.SELF^Referred^Self^^^^|MARKO^Cain^Bighelmet^^^MD|ER||||PR|WI|N||ER||03|||||||||||||||||||ABCDE^|back pain|REG|||201507222358
AL1|1|DA|F000004181^tramadol HCl^From ULTRAM|MO|JERKING|20140525
ACC|20150722^|11
GT1|1||BARTON^CLINTON^FRANCIS^^^||9 S.H.E.I.L.D Training Facility^^CARTERSVILLE^GA^30120^USA^^^BARTOW|(000)000-0000||19510121|M||SA|000-00-0000|||DISABLED|DISABLED^^DISABLED^GA^30092|(000)000-0000||

## Split the file on the "^" and print output

In [4]:
def printJsonSecondLevel(msg1,msg2):
    #msg1 = 'WILSON^Wade^Deadpool^^^MD'
    #msg2 = 'Pv1_1 Value6'
    delim1 = "^"
    result1 = msg1.split(delim1)
    if len(result1) == 1: return "\"" + msg1 + "\"" 
    dataToReturn = "{  " 
    printcomma1 = 0
    for i in range(0,len(result1)):
        if result1[i] != "":
            if printcomma1 == 0:
                dataToReturn += "\"" + msg2 + " Comp" + str(i+1) + "\" : \"" + result1[i] + "\""
                printcomma1 = 1
            else:
                dataToReturn += ", \"" + msg2 + " Comp" + str(i+1) + "\" : \"" + result1[i] + "\""
    dataToReturn += "   }"
    return dataToReturn

## Split the file on the "|" and print output

In [8]:
def printJsonFirstLevel(msg):
    delim = "|"
    result= msg.split(delim)
    if result[0] =="": return msg
    msgheader = result[0] + "_" + result[1]
    if result[0] == "MSH": msgheader = "MSH"
    stringtoprint = "{ \n\t" + msgheader + ": {\n"
    printcomma = 0
    for x in range(2,len(result)):
        if result[x] != "":            
            msgkey = msgheader + " Value" + str(x-1)
            if printcomma == 0:
                #stringtoprint += "\t\t" + "\"" + msgheader + " Value" + str(x-1) + "\" : \"" + result[x] + "\"" 
                stringtoprint += "\t\t" + "\"" + msgkey  + "\" : " + printJsonSecondLevel(result[x],msgkey)
                printcomma = 1
            else:
                #stringtoprint += ", \"" + msgheader + " Value" + str(x-1) + "\" : \"" + result[x] + "\"" 
                stringtoprint += ",  " + "\"" + msgkey  + "\" : " + printJsonSecondLevel(result[x],msgkey)
            
    stringtoprint += "\n\t}\n}"
    return stringtoprint

## Read the file and call the various functions to print the data out

In [9]:
segments = []
segments = data.split('\n')

#for row in data:
    #row1 = row.split("\n")
    #segments.append(row)

#segments

In [10]:
for dataflow in segments:
    print printJsonFirstLevel(dataflow)

{ 
	MSH: {
		"MSH Value2" : "ABCDE",  "MSH Value5" : "201507230021",  "MSH Value7" : {  "MSH Value7 Comp1" : "ADT", "MSH Value7 Comp2" : "A08"   },  "MSH Value8" : "CAGTADM.1.10532994",  "MSH Value9" : "P",  "MSH Value10" : "2.1"
	}
}
{ 
	EVN_A08: {
		"EVN_A08 Value1" : "201507230021",  "EVN_A08 Value4" : {  "EVN_A08 Value4 Comp1" : "R.CA.NAH", "EVN_A08 Value4 Comp2" : "HALL", "EVN_A08 Value4 Comp3" : "NANCY", "EVN_A08 Value4 Comp4" : "A"   }
	}
}
{ 
	PID_1: {
		"PID_1 Value2" : "V000042610",  "PID_1 Value3" : "M88604",  "PID_1 Value4" : {  "PID_1 Value4 Comp1" : "BARTON", "PID_1 Value4 Comp2" : "CLINTON", "PID_1 Value4 Comp3" : "FRANCIS"   },  "PID_1 Value6" : "19510121",  "PID_1 Value7" : "M",  "PID_1 Value8" : {     },  "PID_1 Value9" : "W",  "PID_1 Value10" : {  "PID_1 Value10 Comp1" : "9 E CARTER ST APT A", "PID_1 Value10 Comp3" : "CARTERSVILLE", "PID_1 Value10 Comp4" : "GA", "PID_1 Value10 Comp5" : "30120", "PID_1 Value10 Comp6" : "USA", "PID_1 Value10 Comp9" : "BARTOW"   },  "PI